In [3]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import pickle as p
import os
import sys
import argparse
import time
import random
import pdb
import utils
import bcolz
import numpy as np
import visdom
import matplotlib.pyplot as plt
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader
from Loader4Text import Loader4Text
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [4]:
glovepath='C:\CPSC532P\CPSC532P_A2\glove_pretrained\glove.twitter.27B'
vectors = bcolz.open(f'{glovepath}/27B.50d.dat')[:]
words = p.load(open(f'{glovepath}/27B.50_words.pkl', 'rb'))
word2idx = p.load(open(f'{glovepath}/27B.50_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

In [5]:
datadir='data/'
data_loader=Loader4Text(datadir)

Data Statistics
['negative'] : 441 Distribution: 6.68 %
['neutral'] : 2089 Distribution: 31.66 %
['positive'] : 2954 Distribution: 44.77 %
['verynegative'] : 141 Distribution: 2.14 %
['verypositive'] : 973 Distribution: 14.75 %


In [6]:
embed_matrix = defaultdict(list)
word2embedding=defaultdict(int)
count=0
for i,word in enumerate(data_loader.word2id.keys()):
    try:
        embed_matrix[data_loader.word2id[word]]=glove[word]
    except:
        embed_matrix[data_loader.word2id[word]]=None
        count+=1
    
    
print("{} words not found and replaced with None out of {} words".format(count,len(data_loader.word2id.keys())))   

2311 words not found and replaced with None out of 12192 words


In [7]:
train_data=data_loader.train_data
dev_data=data_loader.dev_data
vocab=data_loader.vocab
word2id=data_loader.word2id
class2id=data_loader.class2id

In [8]:
#embedding_dim=300
hidden_size=200
n_layers=1
input_size=50
batch_size=1

In [9]:
stuff= utils.create_dataset(train_data, word2id, class2id, batch_size=batch_size)

In [11]:
rnn = nn.LSTM(input_size, hidden_size , n_layers)
linear=nn.Linear(hidden_size,5)
softmax=nn.Softmax()

In [12]:

def forward(seq):

    h0=torch.randn(1, 1, hidden_size)
    c0=torch.randn(1, 1, hidden_size)
    output, (hn,cn)=rnn(torch.autograd.Varable(seq),(h0,c0))
                        
    return output,hn,cn    
        

In [13]:
se=[]
tg=[]
for i,(batch, target, lengths , raw) in enumerate(stuff):
    se.append(np.array(batch))
    tg.append(np.array(target))
    

In [14]:
torch.Tensor(tg[1:2]).long().reshape(1,)

tensor([4])

In [15]:
embedded_sequences=[]
targets=[]
for s,t in list(zip(se,tg)):   
    seq= torch.zeros(19, 1, 50)
    for i,index in enumerate(np.nditer(s)):
        if embed_matrix[int(index)] is not None:
            seq[i,0,:]=torch.Tensor(embed_matrix[int(index)])
        else:
            seq[i,0,:]=torch.randn(50)

    embedded_sequences.append(seq)
    targets.append(t)
    

In [19]:
params=list(rnn.parameters())+list(linear.parameters())
optimizer = optim.SGD(params, lr=0.001)

In [16]:
critereon = nn.CrossEntropyLoss()

In [17]:
s=(list(zip(embedded_sequences,targets))[1:5])

In [37]:

num_epochs=1
loss_list=[]
acc_lost=[]
s=(list(zip(embedded_sequences,targets)))
for epoch in range(num_epochs):
    random.shuffle(s)
    total_loss = 0
    y_true = list()
    y_pred = list()
    for i,(seq,tar) in enumerate(s):
        output,hn,cn=forward(seq)
        output=linear(output[-1])
        loss=critereon(output,torch.Tensor(tar).long().reshape(1,))
        pred=softmax(output)
        print(pred)
        pred_index=torch.topk(pred,1)
        print("********" )
        print(pred_index[1],tar)
        y_pred+=list(pred_index.int())
        y_true+=list(torch.Tensor(tar).int())
        print(y_pred)
        print(y_true)
        rnn.zero_grad()
        linear.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss+=loss

        if (i%50 == 0):
            print ("{%d} Epoch:  {%d} Loss {%.2f}" %(i,epoch,loss))
            
    acc = accuracy_score(y_true, y_pred)
    print("Train loss: {%.1f} accuracy: {%.2f} " %(total_loss,100*acc))
    loss_list.append(int(total_loss.data))
    acc_lost.append(acc)
    plt.plot(loss_list)
    plt.show()
    
    
    
    

C:\Users\monst\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


tensor([[0.0622, 0.0767, 0.2835, 0.1284, 0.4492]], grad_fn=<SoftmaxBackward>)
********
tensor([[4]]) [3]


AttributeError: 'torch.return_types.topk' object has no attribute 'int'

In [33]:
acc = accuracy_score(y_true, y_pred)
acc

0.4608

In [25]:
y_true=[1,2,3,4]
y_pred=[3,2,3,4]
acc = accuracy_score(y_true, y_pred)
acc

0.75

In [417]:
len(loss_list)


4

AttributeError: 'list' object has no attribute 'nparray'

In [425]:
[print(int(s.data)) for s in loss_list]

8055
7902
7753
7604


[None, None, None, None]

In [311]:
embedded_sequences[1]

tensor([[[ 9.6343e-03,  8.0978e-01,  2.5799e-01,  4.6127e-01, -2.1069e-01,
          -1.2600e+00,  4.8986e-01,  1.5232e+00,  1.8350e-01,  5.5022e-01,
           1.6149e+00, -3.7828e-01, -4.4425e-01, -3.7330e+00, -6.4956e-01,
          -5.7753e-01,  8.2966e-01,  6.0816e-01, -4.2534e-01, -1.2204e+00,
          -4.7317e-01, -2.9630e-01, -6.7842e-02, -1.2314e-01,  9.2534e-01,
          -6.0226e-01, -3.0520e-01,  3.2710e-01, -8.2888e-01,  8.3770e-01,
          -9.0000e-01, -1.7604e-01,  6.5902e-01,  3.7897e-02, -2.7005e-01,
          -9.8283e-01,  1.2132e-02,  1.5047e-01,  2.2957e-01, -1.6381e-01,
          -1.7096e-01, -1.0970e+00, -4.2669e-01,  4.8125e-01,  8.9528e-01,
          -1.6685e-01, -2.4066e-01, -1.7592e-01,  1.4570e-01, -1.8108e-01]],

        [[ 1.4337e+00, -1.2279e-01,  7.7120e-03,  9.2990e-02, -3.0361e-01,
          -4.5747e-01,  2.8898e-03, -2.4598e-01,  5.7165e-01, -5.9105e-01,
          -4.3149e-01,  1.0901e+00,  1.3153e+00, -3.2484e+00, -3.7211e-01,
           3.0854e-01,

In [312]:
embedded_sequences[2]

tensor([[[-1.1043e+00,  6.0130e-02,  6.3978e-01,  7.2208e-01, -2.7346e-01,
          -2.5681e-01,  3.2445e-01,  1.2919e+00, -4.4900e-01, -2.7585e-02,
          -5.9926e-01, -3.4859e-01,  3.7355e-01, -4.7851e+00,  4.7323e-01,
          -3.0385e-01, -4.0214e-02, -1.2861e-01, -9.5570e-01, -4.9035e-01,
          -1.3257e-01,  1.0362e-01, -2.7648e-01,  5.3580e-01,  2.4512e-01,
          -3.6722e-01,  1.0618e+00,  2.8153e-01,  7.4959e-02, -4.1870e-02,
           8.7498e-02, -3.9773e-01, -1.3448e-01, -3.6708e-01, -3.9039e-02,
           9.2739e-02,  7.9324e-01, -5.6426e-01,  9.6574e-01,  2.0961e-01,
           8.9154e-02, -1.0754e-02,  1.5380e-01,  2.5154e-03, -1.7327e-01,
           9.3592e-01, -3.1894e-01, -3.8511e-01,  3.9943e-01, -9.4465e-02]],

        [[-3.2499e-01,  3.8045e-01,  1.2982e+00, -8.9593e-01,  8.1460e-01,
           1.3381e-01, -2.8355e-01,  1.2501e+00, -1.2337e+00, -4.1691e-01,
           1.0198e-01,  4.1612e-01,  9.1171e-01, -2.0537e+00,  1.5948e-01,
          -1.0529e+00,

In [327]:
output,hn,cn=forward(embedded_sequences[9])

In [329]:
hn[-1]

tensor([[ 0.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00, -0.0000e+00,
         -5.7453e-44, -0.0000e+00,  0.0000e+00, -1.5448e-37, -0.0000e+00,
         -0.0000e+00, -4.5766e-29,  0.0000e+00, -0.0000e+00, -0.0000e+00,
          2.8026e-45,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         -3.2424e-36, -0.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00,
          0.0000e+00,  7.9734e-43, -0.0000e+00, -0.0000e+00, -0.0000e+00,
          0.0000e+00, -0.0000e+00,  0.0000e+00, -0.0000e+00,  8.2773e-39,
          0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
         -0.0000e+00,  0.0000e+00, -2.5111e-42,  0.0000e+00, -0.0000e+00,
         -5.6052e-45,  0.0000e+00,  0.0000e+00, -3.4544e-36, -0.0000e+00,
          1.0322e-39,  0.0000e+00,  0.0000e+00, -0.0000e+00,  0.0000e+00,
         -1.3896e-25,  0.0000e+00,  1.3003e-38, -0.0000e+00, -0.0000e+00,
         -0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -3.9455e-28,
          0.0000e+00,  0.0000e+00, -1.

In [330]:
output=linear(output[-1])

In [331]:
output

tensor([[ 17.6753,  18.1916,  -8.7154, -25.3943,  -1.8082]],
       grad_fn=<AddmmBackward>)

In [333]:
torch.max(pred,1)[1]

tensor([1])

In [289]:
torch.Tensor(targets[1:2]).long().reshape(1,)

tensor([1])

In [337]:
torch.Tensor(targets[1:2]).int()

tensor([[1]], dtype=torch.int32)

In [290]:
loss=critereon(output,torch.Tensor(tg[1:2]).long().reshape(1,))
loss

tensor(1.6006, grad_fn=<NllLossBackward>)

In [291]:
loss.backward()

In [ ]:


test_LSTM=nn.LSTM(input_size,hidden_size,n_layers)
test_linear=nn.Linear()

In [97]:
rnn = nn.LSTM(10, 20, 1)
input = torch.randn(5, 1, 10) #(seq_length,batch,input_dim)
h0 = torch.randn(1, 1, 20)
c0 = torch.randn(1, 1, 20)
output, (hn, cn) = rnn(input, (h0, c0))


In [98]:
input = torch.randn(5, 1, 10)

In [136]:
input.data

tensor([[[ 0.1980,  0.4356,  1.7624, -0.8226, -1.0509, -0.1554,  0.4116,
           1.5567,  0.4918, -0.8718]],

        [[-1.2169, -0.1910,  0.1949, -0.4242, -2.3046,  0.1882, -0.9248,
          -1.6148, -0.4289,  0.4984]],

        [[ 1.6079, -0.5380,  0.3340,  1.5205, -1.9552,  1.1125, -0.4919,
           1.4937,  0.3846,  0.2458]],

        [[-1.0023,  0.3627, -0.9954, -0.7149,  0.6336, -1.5400, -0.5772,
          -1.6534, -0.5591, -0.6738]],

        [[ 0.1190, -0.8805, -1.4209, -0.1050, -0.5172, -1.0016, -0.0966,
          -0.0252,  0.5464, -0.5168]]])

In [137]:
seq.data

tensor([[[ 9.6343e-03,  8.0978e-01,  2.5799e-01,  4.6127e-01, -2.1069e-01,
          -1.2600e+00,  4.8986e-01,  1.5232e+00,  1.8350e-01,  5.5022e-01,
           1.6149e+00, -3.7828e-01, -4.4425e-01, -3.7330e+00, -6.4956e-01,
          -5.7753e-01,  8.2966e-01,  6.0816e-01, -4.2534e-01, -1.2204e+00,
          -4.7317e-01, -2.9630e-01, -6.7842e-02, -1.2314e-01,  9.2534e-01,
          -6.0226e-01, -3.0520e-01,  3.2710e-01, -8.2888e-01,  8.3770e-01,
          -9.0000e-01, -1.7604e-01,  6.5902e-01,  3.7897e-02, -2.7005e-01,
          -9.8283e-01,  1.2132e-02,  1.5047e-01,  2.2957e-01, -1.6381e-01,
          -1.7096e-01, -1.0970e+00, -4.2669e-01,  4.8125e-01,  8.9528e-01,
          -1.6685e-01, -2.4066e-01, -1.7592e-01,  1.4570e-01, -1.8108e-01]],

        [[ 1.4337e+00, -1.2279e-01,  7.7120e-03,  9.2990e-02, -3.0361e-01,
          -4.5747e-01,  2.8898e-03, -2.4598e-01,  5.7165e-01, -5.9105e-01,
          -4.3149e-01,  1.0901e+00,  1.3153e+00, -3.2484e+00, -3.7211e-01,
           3.0854e-01,

In [ ]:
output.data[4,:,:]

In [ ]:
h0=autograd.Variable(torch.randn(1, 1, 200))

In [231]:
loss = nn.CrossEntropyLoss()
input = torch.randn(1, 5, requires_grad=True)
target = torch.empty(1, dtype=torch.long).random_(5)

In [259]:
target.data.size()

torch.Size([1])

In [258]:
input.data.size()

torch.Size([1, 5])

In [234]:
output = loss(input, target)
output

tensor(2.4009, grad_fn=<NllLossBackward>)

In [367]:
import random
number_list = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70]
print ("Original list : ",  number_list)

random.shuffle(number_list) #shuffle method
print ("List after first shuffle  : ",  number_list)

random.shuffle(number_list)
print ("List after second shuffle : ",  number_list)

Original list :  [7, 14, 21, 28, 35, 42, 49, 56, 63, 70]
List after first shuffle  :  [7, 14, 28, 56, 70, 35, 49, 21, 42, 63]
List after second shuffle :  [7, 21, 49, 63, 28, 42, 70, 56, 35, 14]
